# Vacunacions

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

## Data prep

In [ ]:
# Load data
url = "https://dadescovid.cat/static/csv/catalunya_diari_total_pob.zip"
resp = urlopen(url)
zipfile = ZipFile(BytesIO(resp.read()))
df = pd.read_csv(zipfile.open(zipfile.namelist()[0]), sep=";")

# Column transformation and select
df["DATA"] = df.DATA.astype('datetime64[ns]')
df = df[df.DATA>datetime(2020, 12, 20)]
df = df.sort_values("DATA")

# New columns
df["DOSIS_TOTALS"] = df.VACUNATS_DOSI_1+df.VACUNATS_DOSI_2
df["cum1"] = df.VACUNATS_DOSI_1.cumsum()
df["cum2"] = df.VACUNATS_DOSI_2.cumsum()
df["cum"] = df.DOSIS_TOTALS.cumsum()
df["firstDayWeek"] = df.DATA - df.DATA.dt.weekday * timedelta(days=1)

In [ ]:
df.head()

## Daily

In [ ]:
vac_cols = ["VACUNATS_DOSI_1", "VACUNATS_DOSI_2"]
vd = df.groupby("DATA", as_index=False)[vac_cols].sum()
fig, ax = plt.subplots(figsize=(12,6))
p1 = ax.bar(vd.DATA, vd.VACUNATS_DOSI_1, width=0.5, color = [(0.6, 0.8, 1)], label='1a dosi')
p2 = ax.bar(vd.DATA, vd.VACUNATS_DOSI_2, width=0.5, bottom=vd.VACUNATS_DOSI_1, color=[(0.2, 0.4, 1)], label='2a dosi')
ax.set_title("Vacunacions diaries")
ax.set_ylabel("num dosis")
ax.legend()
plt.show()

In [ ]:
vac_cols = ["VACUNATS_DOSI_1", "VACUNATS_DOSI_2"]
vd = df.groupby("DATA", as_index=False)[vac_cols].sum()
fig, ax = plt.subplots(figsize=(12,6))
p1 = ax.bar(vd.DATA, vd.VACUNATS_DOSI_1, width=0.5, color = [(0.6, 0.8, 1)])
p2 = ax.bar(vd.DATA, vd.VACUNATS_DOSI_2, width=0.5, bottom=vd.VACUNATS_DOSI_1, color=[(0.2, 0.4, 1)])

## Weekly

In [ ]:
vac_cols = ["VACUNATS_DOSI_1", "VACUNATS_DOSI_2", "DOSIS_TOTALS"]
vw = df.groupby("firstDayWeek", as_index=False)[vac_cols].sum()
fig, ax = plt.subplots(figsize=(12,6))
p1 = ax.bar(vw.firstDayWeek, vw.VACUNATS_DOSI_1, width=2, color = [(0.6, 0.8, 1)], label='1a dosi')
p2 = ax.bar(vw.firstDayWeek, vw.VACUNATS_DOSI_2, width=2, bottom=vw.VACUNATS_DOSI_1, color = [(0.2, 0.4, 1)], label='2a dosi')
ax.set_title("Vacunacions setmanals")
ax.set_ylabel("num dosis")
ax.legend()
plt.show()

In [ ]:
cum_cols = ["cum1", "cum2"]
vcw = df.groupby("firstDayWeek", as_index=False)[cum_cols].first()
fig, ax = plt.subplots(figsize=(12,6))
w = 2
p1 = ax.bar(vcw.firstDayWeek+timedelta(days=w/2), vcw.cum1, width=w, color = [(0.6, 0.8, 1)], align='center', label='1a dosi')
p2 = ax.bar(vcw.firstDayWeek-timedelta(days=w/2), vcw.cum2, width=w, color = [(0.2, 0.4, 1)], align='center', label='2a dosi')
ax.set_title("Vacunacions cumulatives setmanals")
ax.set_ylabel("num dosis")
ax.legend()
plt.show()

# Weekly total dosis

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(vw.firstDayWeek, vw.DOSIS_TOTALS, width=2, color = [(0.3, 0.3, 0.3)])
ax.set_title("Vacunacions setmanals")
ax.set_ylabel("num dosis")
plt.show()

In [ ]:
vct = df.groupby("firstDayWeek", as_index=False)["cum"].first()
fig, ax = plt.subplots(figsize=(12,6))
p1 = ax.bar(vct.firstDayWeek, vct.cum, width=w, color = [(0.3, 0.3, 0.3)], align='center')
ax.set_title("Vacunacions cumulatives setmanals")
ax.set_ylabel("num dosis")
plt.show()

In [ ]:
(vw.firstDayWeek + pd.Timedelta(36, unit='w')).to_frame()

In [ ]:
# create a future date df
ftr =  (vw.firstDayWeek + pd.Timedelta(12, unit='w')).to_frame()
ftr['DOSIS_TOTALS'] = round(vw.DOSIS_TOTALS.tail(4).head(3).mean())
ftr = ftr[ftr.firstDayWeek>=vw.firstDayWeek.max()]
ftr["future"] = 1

real = vw[:-1]
real["future"] = 0
# join the future data
df_ftr = pd.concat([real, ftr], ignore_index=True)[["firstDayWeek","DOSIS_TOTALS", "future"]]
df_ftr["cum"] = df_ftr.DOSIS_TOTALS.cumsum()

import matplotlib.transforms as transforms
TOTAL_POB = 7560000
fig, ax = plt.subplots(figsize=(12,6))
ax.set_ylim([0, 1.20*df_ftr.cum.max()])
p1 = ax.bar(df_ftr[df_ftr.future==0].firstDayWeek, df_ftr[df_ftr.future==0].cum, 
            width=w, color = [(0.3, 0.3, 0.3)], align='center')
p2 = ax.bar(df_ftr[df_ftr.future==1].firstDayWeek, df_ftr[df_ftr.future==1].cum, 
            width=w, color = [(0.7, 0.9, 0.7)], align='center')
for k in [10,20,30,40]:
    pob_k = 0.02*TOTAL_POB*k
    ax.axhline(y=pob_k, color=(0.7, 0.7, 0.8))
    trans = transforms.blended_transform_factory(
    ax.get_yticklabels()[0].get_transform(), ax.transData)
    ax.text(1.01,pob_k/ax.get_ylim()[1], "{:.0f}".format(k), color=(0.5, 0.5, 0.7), transform=ax.transAxes, 
        ha="left", va="center")
ax.set_title("Projeccio vacunacions cumulatives setmanals")
ax.set_ylabel("num dosis")
plt.show()

In [ ]:
df_ftr